In [1]:
import pandas as pd 
import datetime

In [2]:
tripleg = pd.read_csv('triplegs.csv')

In [3]:
food_df = pd.read_csv('food_inference.csv', index_col=0)

In [4]:
food_df.drop(columns=['id', 'finished_at'], inplace=True)
food_df

,user_id,started_at,lat,lon,retail_id,retail_lat,retail_lon
0,1,2022-09-07 14:34:13-04:00,30.117576,-81.626461,36037,30.118788,-81.625732
1,1,2022-09-23 12:16:32-04:00,30.167915,-81.631710,168227,30.167103,-81.631638
2,1,2022-09-24 20:52:52-04:00,30.146970,-81.632480,151588,30.147409,-81.632477
3,1,2022-09-30 00:42:04-04:00,30.269802,-81.773450,163914,30.269903,-81.772171
4,1,2022-10-09 12:55:19-04:00,30.117490,-81.626550,36037,30.118788,-81.625732
...,...,...,...,...,...,...,...
35440,12388,2022-10-27 19:34:13-04:00,30.268400,-81.613899,237958,30.268553,-81.613953
35441,12388,2022-10-31 19:12:20-04:00,30.269813,-81.610858,45499,30.270267,-81.610809
35442,12399,2022-10-29 05:57:54-04:00,30.196416,-81.744182,119875,30.196510,-81.744141
35443,12411,2022-10-26 23:04:35-04:00,30.322587,-81.557411,58187,30.321955,-81.556618


In [5]:
tripleg.drop(columns=['id'], inplace=True)
tripleg

,user_id,started_at,finished_at,geom
0,0,2022-09-09 01:01:29-04:00,2022-09-09 01:17:29-04:00,LINESTRING (-81.5065289500000034 30.2162162199...
1,1,2022-09-05 17:52:08-04:00,2022-09-05 17:52:57-04:00,LINESTRING (-81.6268599999999935 30.1161999999...
2,1,2022-09-05 20:41:05-04:00,2022-09-05 20:42:55-04:00,LINESTRING (-81.7111000000000018 30.1017399999...
3,1,2022-09-05 21:38:31-04:00,2022-09-05 21:38:42-04:00,LINESTRING (-81.8334399999999960 30.1061100000...
4,1,2022-09-05 23:31:17-04:00,2022-09-05 23:31:28-04:00,LINESTRING (-81.7179699999999940 30.1011300000...
...,...,...,...,...
413238,12419,2022-10-23 17:03:18-04:00,2022-10-23 17:03:30-04:00,LINESTRING (-81.6054400000000015 30.3792200000...
413239,12419,2022-10-23 17:41:20-04:00,2022-10-23 17:41:30-04:00,LINESTRING (-81.6054306099999991 30.3792133299...
413240,12419,2022-10-26 20:13:01-04:00,2022-10-26 20:18:58-04:00,LINESTRING (-81.6119800000000026 30.3824900000...
413241,12419,2022-10-27 01:06:08-04:00,2022-10-27 01:09:53-04:00,LINESTRING (-81.6119800000000026 30.3824900000...


In [6]:
food_df['started_at'] = pd.to_datetime(food_df['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [7]:
ID_list=food_df['user_id'].unique()
len(ID_list)

2517

In [8]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_food_by_user = food_df[food_df['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for food_related_row in related_food_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > food_related_row[2] - time_window) &
            (related_tripled_by_user['finished_at'] <= food_related_row[2])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom'],
                'stop_lat': food_related_row[3], 'stop_lon': food_related_row[4],
                'retail_id': food_related_row[5], 'retail_lat': food_related_row[6], 'retail_lon': food_related_row[7]}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip', 'stop_lat', 'stop_lon', 'retail_id', 'retail_lat', 'retail_lon'])

In [9]:
trip_df

,user_id,trip_started_at,trip_finished_at,trip,stop_lat,stop_lon,retail_id,retail_lat,retail_lon
0,4,2022-09-05 16:37:54-04:00,2022-09-05 16:38:47-04:00,LINESTRING (-81.4142745999999988 30.3244973000...,30.317045,-81.557200,98948,30.317965,-81.556488
1,4,2022-09-06 13:13:38-04:00,2022-09-06 13:15:57-04:00,LINESTRING (-81.4755320499999982 30.3195834200...,30.320775,-81.489335,217378,30.321608,-81.489418
2,4,2022-09-08 22:39:14-04:00,2022-09-08 22:40:42-04:00,LINESTRING (-81.4955279999999931 30.3301033000...,30.322875,-81.489600,217378,30.321608,-81.489418
3,4,2022-09-09 13:13:30-04:00,2022-09-09 13:15:57-04:00,LINESTRING (-81.4755399999999952 30.3195699999...,30.320775,-81.489347,217378,30.321608,-81.489418
4,4,2022-09-09 13:13:30-04:00,2022-09-09 13:15:57-04:00,LINESTRING (-81.4755399999999952 30.3195699999...,30.319298,-81.474303,4324,30.318197,-81.473534
...,...,...,...,...,...,...,...,...,...
8390,12311,2022-10-31 10:52:35-04:00,2022-10-31 10:53:13-04:00,LINESTRING (-81.5906310099999956 30.3595375999...,30.351942,-81.589150,87300,30.352303,-81.589569
8391,12311,2022-10-31 21:37:12-04:00,2022-10-31 21:42:07-04:00,LINESTRING (-81.5217518799999965 30.2936625500...,30.352328,-81.565397,122498,30.352171,-81.566986
8392,12366,2022-10-28 17:45:58-04:00,2022-10-28 17:47:07-04:00,LINESTRING (-81.6991210000000052 30.4050280000...,30.391939,-81.730192,14499,30.391975,-81.730347
8393,12368,2022-10-24 19:14:27-04:00,2022-10-24 19:16:49-04:00,LINESTRING (-81.5728640600000006 30.1725125299...,30.171608,-81.602115,119832,30.172998,-81.602806


In [10]:
print(len(trip_df['user_id'].unique()))

1016


In [11]:
trip_df.to_csv('food_related_trip.csv')